In [1]:
%load_ext tensorboard
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

2021-11-14 22:25:59.562887: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-14 22:25:59.590508: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-14 22:25:59.591131: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
!rm -rf ./logs/hparam_tuning

In [3]:
from pandas import read_parquet
from data import file

data_train = read_parquet(file.news_articles_cleaned_train)
data_test = read_parquet(file.news_articles_cleaned_test)

In [4]:
from preprocessing.categorical import binarizer


X_train = data_train.text_tokenized_stemmed
y_train = data_train.label

X_test = data_test.text_tokenized_stemmed
y_test = data_test.label



batch_size = 32

y_train = data_train.label
label_binarizer=binarizer(y_train)


x_train = data_train['text_stem']
y_train_bin = label_binarizer.transform(y_train)

x_test = data_test['text_stem']
y_test_bin = label_binarizer.transform(y_test)

train_input = tf.data.Dataset.from_tensor_slices((x_train, y_train_bin)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_input = tf.data.Dataset.from_tensor_slices((x_test, y_test_bin)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
train_input


2021-11-14 22:26:02.441316: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-14 22:26:02.442257: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-14 22:26:02.442917: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-14 22:26:02.443472: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

<PrefetchDataset shapes: ((None,), (None, 9)), types: (tf.string, tf.int64)>

In [5]:
%tensorboard --logdir logs/hparam_tuning

In [6]:
from keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from preprocessing.categorical import binarizer
from models import rnn
from tensorboard.plugins.hparams import api as hp


logdir='logs/hparam_tuning/'
session_num = 0
METRIC_ACCURACY = 'accuracy'

HP_NUM_LAYERS = hp.HParam('num_layers', hp.Discrete([1]))
HP_NUM_RNN_UNITS = hp.HParam('num_rnn_units', hp.Discrete([32, 64, 96]))


def train_test_model(hparams):
    
    learning_rate=0.02
    model_metric = ["accuracy"]
    loss_function = CategoricalCrossentropy()
    optimizer = Adam(learning_rate=learning_rate)
    output_classes = len(label_binarizer.classes_)
    rnn_layers = hparams[HP_NUM_LAYERS]
    rnn_units = hparams[HP_NUM_RNN_UNITS]
    
    
    model, model_name = rnn.build_model(X_train, y_train, output_classes, rnn_num_layers = rnn_layers, rnn_units=rnn_units)
    model.summary()
    model.compile(loss=loss_function, optimizer=optimizer, metrics=model_metric)
    
    history = model.fit(train_input, validation_data=test_input, epochs=5, callbacks=[
        tf.keras.callbacks.TensorBoard(logdir),
        hp.KerasCallback(logdir, hparams),
    ])
    _, accuracy = model.evaluate(x_test, y_test_bin)
    return accuracy

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_LAYERS],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )


def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)



for num_layers in HP_NUM_LAYERS.domain.values:
    for num_rnn_units in HP_NUM_RNN_UNITS.domain.values:
        hparams = {
            HP_NUM_LAYERS: num_layers,
            HP_NUM_RNN_UNITS: num_rnn_units,
        }
        
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run(f"{logdir}{run_name}", hparams)
        session_num += 1



--- Starting trial: run-0
{'num_layers': 1, 'num_rnn_units': 32}
Median sequence length:       : 173
Percentil                     : 0.98)
Cutoff sequence length        : 589
Max sequence length           : 1699
Embedding length              : 589
Vocabulary length             : 162207


Model: "rnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 300)         48662100  
_________________________________________________________________
bidirectional_lstm_0 (Bidire (None, None, 64)          85248     
_________________________________________________________________
global_max_pool (GlobalMaxPo (None, 64)                0         
_________________________________________________________________
prediction (Dense)           (None, 9)                 585       
Total params: 48,747,933
Trainable params: 85,833
Non-trainable params: 48,662,100
_________________________________________________________________


2021-11-14 22:26:38.286854: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-14 22:26:38.286905: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-11-14 22:26:38.289792: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1614] Profiler found 1 GPUs
2021-11-14 22:26:38.495016: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-11-14 22:26:38.495205: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed


Epoch 1/5


2021-11-14 22:26:45.549083: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-14 22:26:47.262710: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


  1/289 [..............................] - ETA: 49:09 - loss: 2.2297 - accuracy: 0.0625

2021-11-14 22:26:50.227369: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-14 22:26:50.227412: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


  2/289 [..............................] - ETA: 4:29 - loss: 2.2232 - accuracy: 0.0625 

2021-11-14 22:26:50.998958: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-11-14 22:26:51.007855: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
2021-11-14 22:26:51.138850: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 13265 callback api events and 13262 activity events. 
2021-11-14 22:26:51.367738: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-11-14 22:26:51.749494: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/hparam_tuning/train/plugins/profile/2021_11_14_22_26_51

2021-11-14 22:26:51.946585: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/hparam_tuning/train/plugins/profile/2021_11_14_22_26_51/frankfurt-tf-26-cpu-2-gpu-1.trace.json.gz
2021-11-14 22:26:52.120288: I tensorflow/core/profiler/rpc/client/save_prof

289/289 [==============================] - 79s 240ms/step - loss: 1.6923 - accuracy: 0.3951 - val_loss: 1.4940 - val_accuracy: 0.4825
Epoch 2/5
289/289 [==============================] - 64s 223ms/step - loss: 1.2807 - accuracy: 0.5586 - val_loss: 1.3535 - val_accuracy: 0.5370
Epoch 3/5
289/289 [==============================] - 64s 223ms/step - loss: 1.0696 - accuracy: 0.6353 - val_loss: 1.2745 - val_accuracy: 0.5768
Epoch 4/5
289/289 [==============================] - 66s 227ms/step - loss: 0.8594 - accuracy: 0.7161 - val_loss: 1.1327 - val_accuracy: 0.6342
Epoch 4/5
289/289 [==============================] - 66s 227ms/step - loss: 0.6873 - accuracy: 0.7711 - val_loss: 1.1483 - val_accuracy: 0.6401
Epoch 5/5
33/33 [==============================] - 7s 204ms/step - loss: 1.3447 - accuracy: 0.6294
